## Lab - Cross-Validation and the Bootstrap

#### Import block

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample

%matplotlib inline
plt.style.use('seaborn-white')

Load data

In [2]:
data_path = 'D:\\PycharmProjects\\ISLR\\data\\'
df = pd.read_csv(f'{data_path}Auto.csv', na_values='?').dropna()

### 5.3.1 Validation Set Approach

In [3]:
t_prop = 0.5
p_order = np.arange(1, 5)

Z = np.zeros(p_order.size)
regr = skl_lm.LinearRegression()

for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(df.horsepower.values.reshape(-1,1))
    X_train, X_test, y_train, y_test = train_test_split(X_poly, df.mpg.ravel(),
                                                        test_size=t_prop, random_state=None)
    pred = regr.fit(X_train, y_train).predict(X_test)
    Z[i-1] = mean_squared_error(y_test, pred)
    
for i in p_order:
    print(f'Degree {i}: {Z[i-1]}')

Degree 1: 23.686289752854822
Degree 2: 20.64373982495456
Degree 3: 22.279014168570974
Degree 4: 41.31331223498282


Important to note every time we rerun the code above, we will get a different estimate
for our mse. However, we can consistently see quadratic is better than linear and cubic is not
an improvement over quadratic. 

### LOO - Cross Validation

In [4]:
p_order = np.arange(1,6)

# LOOCV
regr = skl_lm.LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(df)
scores = list()

for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(df.horsepower.values.reshape(-1,1))
    score = cross_val_score(regr, X_poly, df.mpg, cv=loo, scoring='neg_mean_squared_error').mean()
    scores.append(score)

for i in p_order:
    print(f'Degree {i}: {scores[i-1]*-1}')

Degree 1: 24.231513517929226
Degree 2: 19.24821312448974
Degree 3: 19.334984064079364
Degree 4: 19.424430309616614
Degree 5: 19.033204805230227


### K - fold Cross Validation

In [5]:
folds = 10
p_order = np.arange(1,11)

k_score = list()

regr = skl_lm.LinearRegression()
for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(df.horsepower.values.reshape(-1,1))
    kf_10 = KFold(n_splits=folds, random_state=i, shuffle=True)
    k_score.append(cross_val_score(regr, X_poly, df.mpg, cv=kf_10, scoring='neg_mean_squared_error').mean())

for i in p_order:
    print(f'Degree {i}: {k_score[i-1]*-1}')
    

Degree 1: 24.09767573188306
Degree 2: 19.152417393061626
Degree 3: 19.240196381072412
Degree 4: 19.295685540325014
Degree 5: 19.241536896502474
Degree 6: 18.642636498890003
Degree 7: 18.993172040670554
Degree 8: 19.355107505149824
Degree 9: 19.378512199966202
Degree 10: 18.917581991991067


Unfortunately, we dont get the exact results as the book but close enough.

### Bootstrap

In [6]:
df1 = pd.read_csv(f'{data_path}Portfolio.csv', usecols=([1,2]))

Apparently, sklearn method of boostrap is a bit tricky! But someone has already 
solved on this issue (I love you!).
The sklearn-boostrap code is [here](https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/)

In [7]:
def alpha_fn(data):
    X=data['X']
    Y=data['Y']
    alpha = (np.var(Y) - np.cov(X,Y)[0][1])/(np.var(X) + np.var(Y) - 2 * np.cov(X,Y)[0][1])
    return alpha
alpha_fn(df1)

0.5766511516104116

In [8]:
i = 0
alpha = list()
while i < 2001:
    df2 = resample(df1, n_samples=100, replace=True)
    alpha.append(alpha_fn(df2))
    i += 1

In [9]:
print(np.std(alpha))
print(np.mean(alpha))
    

0.09092511557937101
0.5779921539749568


This is the closest that we get for the boostrap method. Notice that I never get anything close 
to .088 only 0.89 and above!

### Bootstrap for Linear Model

In [10]:
df3 = df[['horsepower', 'mpg']]

In [30]:
i = 0
para = []
intercept = []
regr = skl_lm.LinearRegression()
while i < 10001:
    df4 = resample(df3, n_samples=392, replace=True)
    X = df4.horsepower.values.reshape(-1,1)
    y = df4.mpg
    para.append(regr.fit(X, y).coef_[0])
    intercept.append(regr.intercept_)
    i +=1

0.8568529607571254
0.00743389614749305


In [31]:
print(np.std(intercept))
print(np.std(para))

0.8568529607571254
0.00743389614749305


Running the 10000 variation of the data set gives us an extremely similar to the result in the book
. The computational power needed is large! 

Comparing the result using OLS model

In [34]:
import statsmodels.formula.api as smf

mod_fit = smf.ols('mpg~horsepower', data=df3).fit()
print(mod_fit.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145


We got 0.006 which is smaller than 0.0074 - bootstrap estimate.

Fitting the quadratic model and boostrap estimates.

In [37]:
from sklearn.preprocessing import PolynomialFeatures

i = 0
b_1 = []
b_2 = []
intercept = []
regr = skl_lm.LinearRegression()
poly = PolynomialFeatures(2)
while i < 1001:
    df4 = resample(df3, n_samples=392, replace=True)
    X_poly = poly.fit_transform(df4.horsepower.values.reshape(-1,1))
    y = df4.mpg
    b_2.append(regr.fit(X_poly, y).coef_[0])
    b_1.append(regr.fit(X_poly, y).coef_[1])
    intercept.append(regr.intercept_)
    i +=1

In [38]:
print(np.std(intercept))
print(np.std(b_1))
print(np.std(b_2))

1.9959175997066796
0.031917034723871134
0.0


Wow! The estimated standard deviation for horsepower^2 was very closed to 0 or 0 in this case.

Lastly, we take one more look at the OLS model

In [39]:
mod_fit = smf.ols('mpg ~ horsepower + np.power(horsepower, 2)', data=df3).fit()
print(mod_fit.summary().tables[1])

                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  56.9001      1.800     31.604      0.000      53.360      60.440
horsepower                 -0.4662      0.031    -14.978      0.000      -0.527      -0.405
np.power(horsepower, 2)     0.0012      0.000     10.080      0.000       0.001       0.001
